In [ ]:
### movement descriptor
## Hockey Training

import cv2
import numpy as np
import os
import math
from PIL import Image
import csv 
import random

#하키영상 경로
path_ = '/Users/leedagyeong/Desktop/SW/HockeyFights'
video_len = len(os.listdir(path_)) #총 영상 갯수

#프레임 사이즈
#나중에 폭력 vs 비폭력 분류에 feature수와 연관
row_len = 720 
col_len = 1280

row_sub = int(row_len/15) #프레임 세로/15
col_sub = int(col_len/15) #프레임 가로/15
#row_sub * col_sub이 featrue의 수가 될 것

##최종적으로 각 컷마다 Motion Binary Map 저장
MBM_means = np.zeros((row_sub,col_sub,1000))
#ex) MBM_means[:,:,0] : 첫번째 컷의 움직임 서술자



#######################################################################################
#video 거의 41frame, 몇개만 40 or 42
#
#li = list()
#
#for video in range(video_len):
#
#    #path = '/Users/leedagyeong/SW-Maestro/2. Movie to Image/video/팽귄 cut/5'
#    
#    #video번째 컷이 들어있는 파일의 경로
#    path = path_+'/'+os.listdir(path_)[video]
#    
#    #video번째 영상 불러오기
#    cap = cap = cv2.VideoCapture(path)
#    
#    #video번째 영상이 총 몇컷인가
#    cnt=0
#    while(1):
#        ret, frame = cap.read()
#        if ret == False:
#            break
#        cnt += 1
#        
#    li.append(cnt)
#######################################################################################
    
    

list_len = 41 #프레임 갯수


for video in range(1000):
    
    if index % 50 == 0:
        print(index)
    
    #video번째 컷이 들어있는 파일의 경로
    path = path_+'/'+os.listdir(path_)[video]
    
    #video번째 영상 불러오기
    cap = cv2.VideoCapture(path)
    
    #해당 영상의 프레임들 다 모으기
    frames = np.zeros((row_len,col_len,list_len))

    for i in range(41):
        ret, frame = cap.read() #프레임 하나씩
        
        if ret == False:
            break
            
        frame_ = Image.fromarray(frame) #사이즈 변경
        frame_reshape = frame_.resize((col_len, row_len))        
        frame_reshape_ = np.array(frame_reshape)
        frames[:,:,i] = cv2.cvtColor(frame_reshape_,cv2.COLOR_BGR2GRAY) #회색으로 바꿔서 저장
        
    #각각 프레임들의 flow(40flow : 720x1280x2x40)
    flows = np.zeros((row_len,col_len,2,list_len-1))

    for i in range(list_len-1):
        flows[:,:,:,i] = cv2.calcOpticalFlowFarneback(frames[:,:,i],frames[:,:,i+1], None, 0.5, 3, 15, 3, 5, 1.2, 0) 

    #flow에 따른 mag, ang (40장 : 720x1280x40)
    mags = np.zeros((row_len,col_len,list_len-1))
    angs = np.zeros((row_len,col_len,list_len-1))

    for i in range(list_len-1):
        mags[:,:,i], angs[:,:,i] = cv2.cartToPolar(flows[:,:,0,i], flows[:,:,1,i])

    #이전 프레임과의 크기 및 방향의 변화량 : Ds (48x85x39)
    Ds = np.zeros((row_sub,col_sub,list_len-2))

    #15x15인 서브영역
    sub_mags = np.zeros((15,15,list_len-1))
    sub_angs = np.zeros((15,15,list_len-1))

    #이전 프레임과의 크기 및 방향의 변화량
    w = 0.4 #weight

    for a in range(row_sub):
        for b in range(col_sub):
            for i in range(list_len-1):
                sub_mags[:,:,i] = mags[15*a:15*(a+1),15*b:15*(b+1),i] 
                sub_angs[:,:,i] = angs[15*a:15*(a+1),15*b:15*(b+1),i]

            M = sub_mags[7,7,:]
            V = sub_angs[7,7,:]
            
            for j in range(len(M)-1):
                Ds[a,b,j] = (w*abs(M[j+1]-M[j]))+((1-w)*math.acos(V[j+1]*V[j]/abs(V[j+1]*V[j])))/math.radians(180) 
            Ds[np.isnan(Ds)]=0    


    #Motion Binary Map : 움직임 변화가 큰 부분 추출
    
    th = 2.5 #Ds가 th보다 큰 부분만

    MBM = np.zeros_like(Ds)
    MBM[Ds>th]=1
    
    #한 영상 단위 평균
    MBM_mean = MBM.mean(axis=2)

    #모든 천개의 영상 MBM_mean 저장
    MBM_means[:,:,video] = MBM_mean

    

In [ ]:
#엑셀에 저장
f = open('HockeyTraining.csv', 'w', encoding='utf-8', newline='')
wr = csv.writer(f)
for i in range(1000):    
    li = list(MBM_means[:,:,i].reshape(MBM_means.shape[0]*MBM_means.shape[1]))    
    wr.writerow(li)
f.close()

##나중에 엑셀에서 위에서 500개 폭력, 아래 500개 비폭력 셀 추가